# 뉴스 제목으로 기사 분류하기 _ 허채범

# 출처

데이터 : https://dacon.io/competitions/official/235747/data  
한국어 불용어1 : https://deep.chulgil.me/hangugeo-bulyongeo-riseuteu/  
한국어 불용어2 : https://www.ranks.nl/stopwords/korean

In [88]:
import pandas as pd
import numpy as np
import re

import seaborn as sns
import matplotlib.pyplot as plt

# 그래프 글씨체
import platform
if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')

from sklearn.metrics import accuracy_score, log_loss
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from konlpy.tag import Okt, Kkma, Komoran

import warnings 
warnings.filterwarnings(action='ignore')

## 1. 데이터 관련
---

In [89]:
# 데이터 로딩
train = pd.read_csv('./data/train_data.csv', encoding='euc-kr')
test = pd.read_csv('./data/test_data.csv', encoding='euc-kr')
topic = pd.read_csv('./data/topic_dict.csv', encoding='euc-kr')

In [90]:
# 훈련 데이터 확인 (결측치x)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45654 entries, 0 to 45653
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      45654 non-null  int64 
 1   title      45654 non-null  object
 2   topic_idx  45654 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.0+ MB


In [91]:
# train의 기사 제목(title) 확인
train.head()

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4


1) 제목이다 보니, 명사화되어 끝나는 단어들이 많음 => 단어 단위로 토큰화  
2) '...'가 많이 들어감 => 불용어 처리  
3) 영어나 한자의 경우, 국가나 국제 단체를 의미하는 경우가 대다수 => 영어나 한자의 변환(고민)  
4) 뉴스의 제목이다 보니, 비교적 신조어나 비속어 같은 것들은 없음 => 크게 신경x  
5) 제목이라서, 전체적 내용의 핵심을 담고 있기 때문에 분류가 잘 되지 않을까 생각  

In [92]:
# test의 기사 제목(title) 확인
test.head()

,index,title
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사
2,45656,내년부터 국가RD 평가 때 논문건수는 반영 않는다
3,45657,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4,45658,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간


## 2. 텍스트 전처리

### 2-1) 전처리

In [93]:
# 한글만 남기는 함수
def extract_word(text):
    hangul = re.compile('[^가-힣]') 
    result = hangul.sub(' ', text) 
    return result

# 불용어 사전
with open('data/stopwords_3.txt', 'r') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")

# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

train['title'] = train['title'].apply(extract_word).apply(remove_stopwords)
test['title'] = test['title'].apply(extract_word).apply(remove_stopwords)

In [94]:
train

,index,title,topic_idx
0,0,인천 핀란드 항공기 결항 휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다 구글 조원 들여 전역 거점화,4
2,2,외무 긴장완화 해결책은 미국이 경제전쟁 멈추는,4
3,3,클린턴 측근 기업 특수관계 조명 공과 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결,4
...,...,...,...
45649,45649,금융 미국 스티펠과 제휴 선진국 시장 공략,1
45650,45650,보 서울시교육청 신종코로나 확산에 개학 연기 휴업 검토,2
45651,45651,게시판 키움증권 키움 영웅전 실전투자대회,1
45652,45652,답변하는 배기동 국립중앙박물관장,2


### 2-2) 데이터 분리

In [95]:
from sklearn.model_selection import train_test_split

In [96]:
X_train, X_test, y_train, y_test = train_test_split(train['title'], train['topic_idx'], random_state=42,
                                                    stratify=train['topic_idx'], test_size=0.2)

### 2-3) 토큰화

In [97]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
# tokenizer.fit_to_text(sentenceList)
tokenizer.fit_on_texts(X_train)

In [98]:
len(tokenizer.word_counts)

65765

In [99]:
seq_train = tokenizer.texts_to_sequences(X_train)
seq_test = tokenizer.texts_to_sequences(X_test)

In [100]:
pad_train = pad_sequences(seq_train)
pad_test = pad_sequences(seq_test)

In [101]:
pad_train.shape, pad_test.shape

((36523, 12), (9131, 12))

In [102]:
myToken = Tokenizer(num_words=len(tokenizer.word_counts))
myToken.fit_on_texts(X_train)

In [103]:
seq_train = myToken.texts_to_sequences(X_train)
seq_test = myToken.texts_to_sequences(X_test)

pad_train = pad_sequences(seq_train)
pad_test = pad_sequences(seq_test)

## 3. 모델링 ( modeling )

In [114]:
from tensorflow.keras.layers import Embedding, Dense, LSTM, GRU, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

vocab_size = 10
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(65765, 10, input_length=12))
model.add(LSTM(10))
model.add(Dense(7, activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [115]:
history = model.fit(pad_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
229/229 [==============================] - 4s 10ms/step - loss: 1.7473 - acc: 0.3189 - val_loss: 1.3649 - val_acc: 0.4969
Epoch 2/10
229/229 [==============================] - 2s 7ms/step - loss: 1.0771 - acc: 0.6257 - val_loss: 1.1020 - val_acc: 0.6327
Epoch 3/10
229/229 [==============================] - 2s 8ms/step - loss: 0.6819 - acc: 0.8069 - val_loss: 0.9682 - val_acc: 0.6916
Epoch 4/10
229/229 [==============================] - 2s 7ms/step - loss: 0.4306 - acc: 0.8963 - val_loss: 0.9793 - val_acc: 0.6980
Epoch 5/10
229/229 [==============================] - 2s 7ms/step - loss: 0.2804 - acc: 0.9392 - val_loss: 0.9646 - val_acc: 0.7173
Epoch 6/10
229/229 [==============================] - 2s 7ms/step - loss: 0.1833 - acc: 0.9640 - val_loss: 1.0077 - val_acc: 0.7207
Epoch 7/10
229/229 [==============================] - 1s 6ms/step - loss: 0.1255 - acc: 0.9766 - val_loss: 1.0467 - val_acc: 0.7195
Epoch 8/10
229/229 [==============================] - 2s 7ms/step - loss: 0

In [116]:
# 평가
model.evaluate(pad_train,y_train)

1142/1142 [==============================] - 1s 870us/step - loss: 0.2677 - acc: 0.9395


[0.26768559217453003, 0.9395175576210022]

In [117]:
predicted = model.predict(pad_test)
print(predicted[0].argmax())

1


In [118]:
y_test

6262     1
20584    6
40627    4
20280    4
15709    3
        ..
31650    3
38446    0
41597    2
1721     3
9581     6
Name: topic_idx, Length: 9131, dtype: int64

In [124]:
# 실제값 예측
pre_txt = ["늘어나는 부동산 '수상한 직거래'... 아파트도 반값 세일?", 
           "33홈런 KT 박병호, 오른 발목 부상으로 시즌 아웃 유력... '복귀 힘들 듯'",
           "코스피, '2년 2개월만에 역대최저' 2160대... 환율 1440원 돌파"
          ]

pre_txt1 = ['사회(2)','스포츠(5)','경제(1)']

In [125]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = [word for word in new_sentence if not word in stopwords]
    encoded = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(encoded, maxlen = 12)

    score = (model.predict(pad_new))
    score = score.argmax()
    print(f"예측은 {score}입니다.")

In [126]:
tokenizer = Tokenizer()

In [127]:
for n in range(len(pre_txt)):
    sentiment_predict(pre_txt[n])
    print(f'정답은 {pre_txt1[n]}입니다\n')

예측은 2입니다.
정답은 사회(2)입니다

예측은 2입니다.
정답은 스포츠(5)입니다

예측은 2입니다.
정답은 경제(1)입니다



In [123]:
topic.T

,0,1,2,3,4,5,6
topic,IT과학,경제,사회,생활문화,세계,스포츠,정치
topic_idx,0,1,2,3,4,5,6
